In [3]:
# IMPORT STATEMENTS
import sys
sys.path.append("/Users/rohan/public_html/Hegemon")
import StepMiner as smn
import HegemonUtil as hu
import re
import numpy as np
import scipy
import io
from scipy import io
import math
import itertools
from itertools import combinations 
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import statsmodels.stats.proportion
import seaborn as sns
import scanpy as sc
import GEOparse

#### GSE139544: ARID3a expression in human hematopoietic stem cells is associated with distinct gene patterns in aged individuals

<br>This notebook demonstrates how to process smaller scRNA-seq datasets such as GSE139544 (n = 729).
<br>scRNA-seq of hematopoietic stem cells</br>
Paper which collected data: https://immunityageing.biomedcentral.com/articles/10.1186/s12979-020-00198-6
Analyzed using Boolean networks: https://www.sciencedirect.com/science/article/pii/S2001037021003974
<br></br>
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE138544

In [6]:
# Get data
aged_df = pd.DataFrame(pd.read_csv('GSE138544_Partek_michelle_aged_Normalization_Normalized_counts.txt', sep = '\t', header=0))
young_df = pd.DataFrame(pd.read_csv('GSE138544_Partek_michelle_young_Normalization_Normalized_counts.txt', sep = '\t', header=0))

In [22]:
# Since all the values are under 20, we confirm that it is log normalized and log2 has been taken.
# Paper says that Log normalized counts per millions (CPM) were log2-transformed.
print(max(aged_df.max(numeric_only=True)), max(young_df.max(numeric_only=True)))

19.8205 19.7457


In [47]:
# Make expr file
expr = pd.merge(aged_df, young_df, on="Feature")
expr = expr.rename(columns={'Feature': 'Name'})
genes = pd.DataFrame()
genes["ProbeID"] = expr["Name"]
expr = pd.concat([genes, expr], axis=1)
expr
# We still need to change the column names from the sample clinical names to GSM IDs.

,ProbeID,Name,1_Aged_A,1_Aged_C,1_Aged_D,2_Aged_A,2_Aged_B,2_Aged_C,2_Aged_D,3_Aged_A,...,94_Yg_A,94_Yg_B,94_Yg_C,94_Yg_D,95_Yg_A,95_Yg_C,95_Yg_D,96_Yg_A,96_Yg_C,96_Yg_D
0,A1BG,A1BG,0.054748,0.000145,7.307650,5.616870,6.049130,4.701390,0.000144,7.743910,...,0.000146,0.000144,6.101560,0.000145,0.000146,0.000145,1.803640,5.779020,0.000145,0.000145
1,A1BG-AS1,A1BG-AS1,1.431330,0.000145,0.000145,0.000145,0.000145,0.000144,0.000144,0.000145,...,0.000146,0.000144,0.000145,0.000145,0.000146,0.000145,0.000144,0.000145,0.000145,0.000145
2,A1CF,A1CF,0.000145,0.000145,0.000145,0.000145,6.320250,0.000144,0.000144,0.000145,...,0.000146,0.000144,0.000145,0.000145,0.000146,0.000145,0.000144,0.000145,0.000145,0.000145
3,A2M,A2M,0.000145,0.000145,0.000145,0.000145,0.000145,0.000144,0.000144,0.000145,...,0.000146,0.000144,0.000145,0.000145,0.000146,0.000145,0.000144,0.000145,0.000145,0.000145
4,A2M-AS1,A2M-AS1,0.000145,0.000145,0.000145,0.000145,0.000145,0.000144,0.000144,0.000145,...,0.000146,0.000144,0.000145,0.000145,0.000146,0.000145,0.000144,5.713640,0.000145,0.000145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45853,ZYG11A,ZYG11A,0.000145,0.000145,0.000145,0.000145,4.328100,0.000144,0.000144,0.000145,...,0.000146,0.000144,0.000145,6.119100,7.441360,0.000145,0.000144,5.713640,4.481180,7.502960
45854,ZYG11AP1,ZYG11AP1,0.000145,0.000145,0.000145,0.000145,0.000145,0.000144,0.000144,0.000145,...,0.000146,0.000144,0.000145,0.000145,0.000146,0.000145,0.000144,0.000145,0.000145,0.000145
45855,ZYG11B,ZYG11B,0.000145,6.196720,0.000145,7.182100,0.000145,0.000144,0.000144,5.763960,...,7.500130,0.000144,5.243060,0.000145,0.000146,4.641820,1.803640,8.741510,0.000145,0.000145
45856,ZYX,ZYX,0.000145,0.000145,0.000145,0.000145,6.320250,0.000144,0.000144,0.000145,...,5.646760,0.000144,0.000145,0.000145,0.000146,0.000145,0.000144,0.000145,0.000145,0.000145


In [46]:
# This script generates ih and survival files.
%run process.py

25-May-2022 17:31:18 DEBUG utils - Directory ./ already exists. Skipping.
25-May-2022 17:31:18 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE138nnn/GSE138544/soft/GSE138544_family.soft.gz to ./GSE138544_family.soft.gz


-Downloading GSE138544


100%|██████████| 25.5k/25.5k [00:00<00:00, 59.2kB/s]
25-May-2022 17:31:19 DEBUG downloader - Size validation passed
25-May-2022 17:31:19 DEBUG downloader - Moving /tmp/tmpnxh7705x to /mnt/Users/rohan/public_html/Hegemon/Data2/Data/GSE138544/GSE138544_family.soft.gz
25-May-2022 17:31:19 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE138nnn/GSE138544/soft/GSE138544_family.soft.gz
25-May-2022 17:31:19 INFO GEOparse - Parsing ./GSE138544_family.soft.gz: 
25-May-2022 17:31:19 DEBUG GEOparse - DATABASE: GeoMiame
25-May-2022 17:31:19 DEBUG GEOparse - SERIES: GSE138544
25-May-2022 17:31:19 DEBUG GEOparse - PLATFORM: GPL24676
25-May-2022 17:31:19 DEBUG GEOparse - SAMPLE: GSM4112821
25-May-2022 17:31:19 DEBUG GEOparse - SAMPLE: GSM4112822
25-May-2022 17:31:19 DEBUG GEOparse - SAMPLE: GSM4112823
25-May-2022 17:31:19 DEBUG GEOparse - SAMPLE: GSM4112824
25-May-2022 17:31:19 DEBUG GEOparse - SAMPLE: GSM4112825
25-May-2022 17:31:19 DEBUG GEOparse - SAMPLE: GSM4112

25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4112954
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4112955
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4112956
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4112957
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4112958
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4112959
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4112960
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4112961
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4112962
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4112963
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4112964
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4112965
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4112966
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4112967
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4112968
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4112969
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4112970
25-May-2022 17:31:20 DEBUG GEOp

25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113098
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113099
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113100
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113101
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113102
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113103
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113104
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113105
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113106
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113107
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113108
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113109
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113110
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113111
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113112
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113113
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113114
25-May-2022 17:31:20 DEBUG GEOp

25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113242
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113243
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113244
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113245
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113246
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113247
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113248
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113249
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113250
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113251
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113252
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113253
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113254
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113255
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113256
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113257
25-May-2022 17:31:20 DEBUG GEOparse - SAMPLE: GSM4113258
25-May-2022 17:31:20 DEBUG GEOp

25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113386
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113387
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113388
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113389
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113390
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113391
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113392
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113393
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113394
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113395
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113396
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113397
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113398
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113399
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113400
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113401
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113402
25-May-2022 17:31:21 DEBUG GEOp

25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113530
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113531
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113532
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113533
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113534
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113535
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113536
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113537
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113538
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113539
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113540
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113541
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113542
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113543
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113544
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113545
25-May-2022 17:31:21 DEBUG GEOparse - SAMPLE: GSM4113546
25-May-2022 17:31:21 DEBUG GEOp

Done processing
Starting survival_ih
array_header ['GSM4112821', 'GSM4112822', 'GSM4112823', 'GSM4112824', 'GSM4112825', 'GSM4112826', 'GSM4112827', 'GSM4112828', 'GSM4112829', 'GSM4112830', 'GSM4112831', 'GSM4112832', 'GSM4112833', 'GSM4112834', 'GSM4112835', 'GSM4112836', 'GSM4112837', 'GSM4112838', 'GSM4112839', 'GSM4112840', 'GSM4112841', 'GSM4112842', 'GSM4112843', 'GSM4112844', 'GSM4112845', 'GSM4112846', 'GSM4112847', 'GSM4112848', 'GSM4112849', 'GSM4112850', 'GSM4112851', 'GSM4112852', 'GSM4112853', 'GSM4112854', 'GSM4112855', 'GSM4112856', 'GSM4112857', 'GSM4112858', 'GSM4112859', 'GSM4112860', 'GSM4112861', 'GSM4112862', 'GSM4112863', 'GSM4112864', 'GSM4112865', 'GSM4112866', 'GSM4112867', 'GSM4112868', 'GSM4112869', 'GSM4112870', 'GSM4112871', 'GSM4112872', 'GSM4112873', 'GSM4112874', 'GSM4112875', 'GSM4112876', 'GSM4112877', 'GSM4112878', 'GSM4112879', 'GSM4112880', 'GSM4112881', 'GSM4112882', 'GSM4112883', 'GSM4112884', 'GSM4112885', 'GSM4112886', 'GSM4112887', 'GSM4112888

In [73]:
# Read ih file
ih_df = pd.DataFrame(pd.read_csv('GSE138544-GPL24676-ih.txt', sep = '\t', header=0))

# This is so we can easily rename expr columns to GSM IDs using information in the ih file.
ih_df["ClinicalHeader"] = ih_df["ClinicalHeader"].apply(lambda x: x.replace("    ", "_"))
gsm_dict = {ih_df["ClinicalHeader"][i]:ih_df["ArrayID"][i] for i in ih_df.index}
expr = expr.rename(columns=gsm_dict)

In [77]:
# Export completed IH file (survival has already been generated)
path_dir='./'
ih_df
ih_df.to_csv(path_dir+"GSE138544-GPL24676-ih.txt", header=True, index=False,sep='\t')

In [78]:
# Export completed expr file
expr
expr.to_csv(path_dir+"GSE138544-GPL24676-expr.txt", header=True, index=False,sep='\t')

In [80]:
# Build idx file

def make_idx():
    print('Starting make_idx')
    expr = path_dir+'GSE138544-GPL24676-expr.txt'

    ptr = []
    ids = []
    name = []
    desc = []
    pos = 0

    with open(expr, 'rb') as f:
        for line in f:
            if pos == 0:
                pos += len(line)
            else:
                ptr.append(pos)
                pos += len(line)
                split = line.decode("utf-8").split('\t')
                ids.append(split[0])
                name.append(split[1].split(':')[0])
                desc.append(':'.join(split[1].split(':')[1:]))
        f.close()

    with open(path_dir+'GSE138544-GPL24676-idx.txt', 'w') as f:
        f.write('ProbeID\tPtr\tName\tDescription\n')
        for i in range(len(ids)):
            f.write('{}\t{}\t{}\t{}\n'.format(ids[i], ptr[i], name[i], desc[i]))
        f.close()
    print("Done with make_idx")
    
make_idx()

Starting make_idx
Done with make_idx


Run gse_processing.sh script generates the remaining files from the four we have generated.
Then copy configurations to your explore.conf file, and processing is complete! 